# ASR Assignment 2022-23

This notebook has been provided as a template to get you started on the assignment.  Feel free to use it for your development, or do your development directly in Python.

You can find a full description of the assignment [here](http://www.inf.ed.ac.uk/teaching/courses/asr/2022-23/coursework.pdf).

You are provided with two Python modules `observation_model.py` and `wer.py`.  The first was described in [Lab 3](https://github.com/ZhaoZeyu1995/asr_labs/blob/master/asr_lab3_4.ipynb).  The second can be used to compute the number of substitution, deletion and insertion errors between ASR output and a reference text.

It can be used as follows:

```python
import wer

my_refence = 'A B C'
my_output = 'A C C D'

wer.compute_alignment_errors(my_reference, my_output)
```

This produces a tuple $(s,d,i)$ giving counts of substitution,
deletion and insertion errors respectively - in this example (1, 0, 1).  The function accepts either two strings, as in the example above, or two lists.  Matching is case sensitive.

## Template code

Assuming that you have already made a function to generate an WFST, `create_wfst()` and a decoder class, `MyViterbiDecoder`, you can perform recognition on all the audio files as follows:


In [1]:
import glob
import os
import wer
import observation_model
import openfst_python as fst

# ... (add your code to create WFSTs and Viterbi Decoder)
import observation_model
import math
import openfst_python as fst
import time

class MyViterbiDecoder:
    
    NLL_ZERO = 1e10  # define a constant representing -log(0).  This is really infinite, but approximate
                     # it here with a very large number
    
    def __init__(self, f, audio_file_name):
        """Set up the decoder class with an audio file and WFST f
        """
        self.om = observation_model.ObservationModel()
        self.f = f
        
        if audio_file_name:
            self.om.load_audio(audio_file_name)
        else:
            self.om.load_dummy_audio()
        
        self.initialise_decoding()

        
    def initialise_decoding(self):
        """set up the values for V_j(0) (as negative log-likelihoods)
        
        """
        
        self.V = []   # stores likelihood along best path reaching state j
        self.B = []   # stores identity of best previous state reaching state j
        self.W = []   # stores output labels sequence along arc reaching j - this removes need for 
                      # extra code to read the output sequence along the best path
        
        for t in range(self.om.observation_length()+1):
            self.V.append([self.NLL_ZERO]*self.f.num_states())
            self.B.append([-1]*self.f.num_states())
            self.W.append([[] for i in range(self.f.num_states())])  #  multiplying the empty list doesn't make multiple
        
        # The above code means that self.V[t][j] for t = 0, ... T gives the Viterbi cost
        # of state j, time t (in negative log-likelihood form)
        # Initialising the costs to NLL_ZERO effectively means zero probability    
        
        # give the WFST start state a probability of 1.0   (NLL = 0.0)
        self.V[0][self.f.start()] = 0.0
        
        # some WFSTs might have arcs with epsilon on the input (you might have already created 
        # examples of these in earlier labs) these correspond to non-emitting states, 
        # which means that we need to process them without stepping forward in time.  
        # Don't worry too much about this!  
        self.traverse_epsilon_arcs(0)        
        
    def traverse_epsilon_arcs(self, t):
        """Traverse arcs with <eps> on the input at time t
        
        These correspond to transitions that don't emit an observation
        
        We've implemented this function for you as it's slightly trickier than
        the normal case.  You might like to look at it to see what's going on, but
        don't worry if you can't fully follow it.
        
        """
        
        states_to_traverse = list(self.f.states()) # traverse all states
        while states_to_traverse:
            
            # Set i to the ID of the current state, the first 
            # item in the list (and remove it from the list)
            i = states_to_traverse.pop(0)   
        
            # don't bother traversing states which have zero probability
            if self.V[t][i] == self.NLL_ZERO:
                    continue
        
            for arc in self.f.arcs(i):
                
                if arc.ilabel == 0:     # if <eps> transition
                  
                    j = arc.nextstate   # ID of next state  
                
                    if self.V[t][j] > self.V[t][i] + float(arc.weight):
                        
                        # this means we've found a lower-cost path to
                        # state j at time t.  We might need to add it
                        # back to the processing queue.
                        self.V[t][j] = self.V[t][i] + float(arc.weight)
                        
                        # save backtrace information.  In the case of an epsilon transition, 
                        # we save the identity of the best state at t-1.  This means we may not
                        # be able to fully recover the best path, but to do otherwise would
                        # require a more complicated way of storing backtrace information
                        self.B[t][j] = self.B[t][i] 
                        
                        # and save the output labels encountered - this is a list, because
                        # there could be multiple output labels (in the case of <eps> arcs)
                        if arc.olabel != 0:
                            self.W[t][j] = self.W[t][i] + [arc.olabel]
                        else:
                            self.W[t][j] = self.W[t][i]
                        
                        if j not in states_to_traverse:
                            states_to_traverse.append(j)

    
    def forward_step(self, t):
        
        forward_computations = 0
          
        for i in self.f.states():
            
            if not self.V[t-1][i] == self.NLL_ZERO:   # no point in propagating states with zero probability
                
                for arc in self.f.arcs(i):
                    
                    if arc.ilabel != 0: # <eps> transitions don't emit an observation
                        forward_computations += 1
                        j = arc.nextstate
                        tp = float(arc.weight)  # transition prob
                        ep = -self.om.log_observation_probability(self.f.input_symbols().find(arc.ilabel), t)  # emission negative log prob
                        prob = tp + ep + self.V[t-1][i] # they're logs
                        if prob < self.V[t][j]:
                            self.V[t][j] = prob
                            self.B[t][j] = i
                            
                            # store the output labels encountered too
                            if arc.olabel !=0:
                                self.W[t][j] = [arc.olabel]
                            else:
                                self.W[t][j] = []
                                
        return forward_computations
                            
    
    def finalise_decoding(self):
        """ this incorporates the probability of terminating at each state
        """
        
        for state in self.f.states():
            final_weight = float(self.f.final(state))
            if self.V[-1][state] != self.NLL_ZERO:
                if final_weight == math.inf:
                    self.V[-1][state] = self.NLL_ZERO  # effectively says that we can't end in this state
                else:
                    self.V[-1][state] += final_weight
                    
        # get a list of all states where there was a path ending with non-zero probability
        finished = [x for x in self.V[-1] if x < self.NLL_ZERO]
        if not finished:  # if empty
            print("No path got to the end of the observations.")
        
        
    def decode(self):
        forward_computations = 0
        initial = time.time()
        self.initialise_decoding()
        t = 1
        while t <= self.om.observation_length():
            forward_computations += self.forward_step(t)
            self.traverse_epsilon_arcs(t)
            t += 1
        self.finalise_decoding()
        final = time.time()
        return (forward_computations, (final - initial))
    
    def backtrace(self):
        
        initial = time.time()
        best_final_state = self.V[-1].index(min(self.V[-1])) # argmin
        best_state_sequence = [best_final_state]
        best_out_sequence = []
        
        t = self.om.observation_length()   # ie T
        j = best_final_state
        
        while t >= 0:
            i = self.B[t][j]
            best_state_sequence.append(i)
            best_out_sequence = self.W[t][j] + best_out_sequence  # computer scientists might like
                                                                                # to make this more efficient!

            # continue the backtrace at state i, time t-1
            j = i  
            t-=1
            
        best_state_sequence.reverse()
        
        # convert the best output sequence from FST integer labels into strings
        
        
        
        # convert the best output sequence from FST integer labels into strings
        best_out_sequence = ' '.join([ self.f.output_symbols().find(label) for label in best_out_sequence])
        
        
        output=[]
        full_words = [x for x in lex.values()]
        
        this_str=[]
        
        for label in best_out_sequence.split(" "):
            this_str.append(label)
            
            if this_str in lex.values():
                for i in  lex:
                    
                    if lex.get(i)==this_str:
                        output+=[f"{i}"]
                this_str=[]
        output = " ".join([n for n in output])
        final = time.time()
        return (best_state_sequence, output, (final-initial))
import openfst_python as fst

def parse_lexicon(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            lex[line[0]] = line[1:]  # first field the word, the rest is the phones
    return lex

def generate_symbol_tables(lexicon, n=3):
    '''
    Return word, phone and state symbol tables based on the supplied lexicon
        
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
        n (int): number of states for each phone HMM
        
    Returns:
        word_table (fst.SymbolTable): table of words
        phone_table (fst.SymbolTable): table of phones
        state_table (fst.SymbolTable): table of HMM phone-state IDs
    '''
    
    state_table = fst.SymbolTable()
    phone_table = fst.SymbolTable()
    word_table = fst.SymbolTable()
    
    # add empty <eps> symbol to all tables
    state_table.add_symbol('<eps>')
    phone_table.add_symbol('<eps>')
    word_table.add_symbol('<eps>')
    
    for word, phones  in lexicon.items():
        
        word_table.add_symbol(word)
        
        for p in phones: # for each phone
            
            phone_table.add_symbol(p)
            for i in range(1,n+1): # for each state 1 to n
                state_table.add_symbol('{}_{}'.format(p, i))
            
    return word_table, phone_table, state_table


# call these two functions
lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)
def generate_phone_wfst(f, start_state, phone, n):
    """
    Generate a WFST representing an n-state left-to-right phone HMM.
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assumed to exist already
        phone (str): the phone label 
        n (int): number of states of the HMM excluding start and end
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    
    for i in range(1, n+1):
    
        in_label = state_table.find('{}_{}'.format(phone, i))
        
        # self-loop back to current state
        f.add_arc(current_state, fst.Arc(in_label, 0, None, current_state))
        
        # transition to next state
        
        # we want to output the phone label on the final state
        # note: if outputting words instead this code should be modified
        if i == n:
            out_label = phone_table.find(phone)
        else:
            out_label = 0   # output empty <eps> label
            
        next_state = f.add_state()
        f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_state))    
       
        current_state = next_state
    return current_state



def create_wfst(n):
    """ Generate a WFST for any word in the lexicon, composed of n-state phone WFSTs.
        This will currently output phone labels.  
    
    Args:
        
        start_state (int): the index of the first state, assumed to exist already
        word (str): the word to generate
        n (int): states per phone HMM
        
    Returns:
        the constructed WFST
    
    """
    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    for word, phones in lex.items():
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, None, current_state))
        
        for phone in phones: 
            current_state = generate_phone_wfst(f, current_state, phone, n)
        # note: new current_state is now set to the final state of the previous phone WFST
        
        f.set_final(current_state)
        f.add_arc(current_state, fst.Arc(0, 0, None, start_state))
    return f




    


def read_transcription(wav_file):
    """
    Get the transcription corresponding to wav_file.
    """
    
    transcription_file = os.path.splitext(wav_file)[0] + '.txt'
    
    with open(transcription_file, 'r') as f:
        transcription = f.readline().strip()
    
    return transcription


f = create_wfst(3)
f.set_input_symbols(state_table)
f.set_output_symbols(phone_table)
print([x for x in lex.values()])

def num_states(f):
    counter = 0
    for x in f.states():
        counter += 1
    return counter

def total_num_arcs(f):
    return sum(f.num_arcs(s) for s in f.states())



    

[['ey'], ['ah', 'v'], ['p', 'eh', 'k'], ['p', 'eh', 'p', 'er', 'z'], ['p', 'iy', 't', 'er'], ['p', 'ih', 'k', 't'], ['p', 'ih', 'k', 'ah', 'l', 'd'], ['p', 'ay', 'p', 'er'], ['dh', 'iy'], ['w', 'eh', 'r', 'z']]


In [2]:
!pip install pandas
import pandas as pd
results_df=[]
import wer

print(num_states(f))
print(total_num_arcs(f))

for wav_file in glob.glob('/group/teaching/asr/labs/recordings/*.wav'):    # replace path if using your own
                                                                           # audio files
    
    decoder = MyViterbiDecoder(f, wav_file)
    
    forward_computations, decode_speed = decoder.decode()
    (state_path, words, backtrace_speed) = decoder.backtrace()  # you'll need to modify the backtrace() from Lab 4
                                               # to return the words along the best path
    
    transcription = read_transcription(wav_file)


    
    error_counts = wer.compute_alignment_errors(transcription, words)
    word_count = len(transcription.split())
    word_count = len(transcription.split())
    worderror = sum(error_counts)/word_count
         # you'll need to accumulate these to produce an overall Word Error Rate
    filename = wav_file.split('/')[-1]
    row = {
        'Filename': filename,
        'Insertions': error_counts[2],
        'Deletions': error_counts[1],
        'Substitutions': error_counts[0],
        'WER': worderror,
        'Word Count': word_count,
        'transcription':transcription,
        'Viterbi transcription':words,
        'Decode speed': decode_speed,
        'Backtrace speed': backtrace_speed,
        'Forward computations': forward_computations
    }
    print(row)
    
    # Append results to the DataFrame
    results_df.append(row)
results_df = pd.DataFrame(results_df)
# Save the DataFrame to a CSV file
results_df.to_csv('results.csv', index=False)


116
230
{'Filename': '0000.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 0, 'WER': 0.23529411764705882, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': "pickled the peter piper picked a the peck of pickled peppers where's the peck of pickled peppers peter piper picked the", 'Decode speed': 6.3865509033203125, 'Backtrace speed': 0.0027070045471191406, 'Forward computations': 240318}
{'Filename': '0001.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.0, 'Word Count': 7, 'transcription': 'peter picked a peck of pickled peppers', 'Viterbi transcription': 'pickled the picked the peter picked the of peck of pickled peppers the', 'Decode speed': 3.92000150680542, 'Backtrace speed': 0.0010421276092529297, 'Forward computations': 136368}
{'Filename': '0002.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.5, 'Word Count': 6, 'transcrip

{'Filename': '0022.wav', 'Insertions': 11, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.5, 'Word Count': 8, 'transcription': 'peter piper piper peter peter peter piper piper', 'Viterbi transcription': 'the picked peck piper the of piper the peter picked peter of peter of piper the of piper the', 'Decode speed': 6.697383165359497, 'Backtrace speed': 0.0013766288757324219, 'Forward computations': 150648}
{'Filename': '0023.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 0, 'WER': 1.0, 'Word Count': 4, 'transcription': 'peter picked piper peppers', 'Viterbi transcription': 'the peter of picked piper of peppers the', 'Decode speed': 3.027592658996582, 'Backtrace speed': 0.0004811286926269531, 'Forward computations': 97938}
{'Filename': '0024.wav', 'Insertions': 6, 'Deletions': 0, 'Substitutions': 2, 'WER': 1.3333333333333333, 'Word Count': 6, 'transcription': 'peter peter peter peter peter peter', 'Viterbi transcription': 'the peter peck peter picked peter peck peter picked pickled

{'Filename': '0045.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 4, 'WER': 1.0, 'Word Count': 6, 'transcription': 'piper peter picked pickled piper peppers', 'Viterbi transcription': "pickled the where's peck pickled picked peppers the", 'Decode speed': 2.916973829269409, 'Backtrace speed': 0.0005931854248046875, 'Forward computations': 89748}
{'Filename': '0046.wav', 'Insertions': 8, 'Deletions': 0, 'Substitutions': 4, 'WER': 1.0909090909090908, 'Word Count': 11, 'transcription': 'peter picked peppers of the pickled peck of peppers of piper', 'Viterbi transcription': "a the the picked the the where's of the of pickled pickled peck where's of of the of the", 'Decode speed': 3.470012903213501, 'Backtrace speed': 0.0005664825439453125, 'Forward computations': 96888}
{'Filename': '0047.wav', 'Insertions': 5, 'Deletions': 1, 'Substitutions': 3, 'WER': 0.5294117647058824, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled p

{'Filename': '0067.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.75, 'Word Count': 8, 'transcription': 'pickled peter peppers pickled peppers piper of piper', 'Viterbi transcription': 'the picked of peter the peppers pickled peppers piper of piper picked the', 'Decode speed': 4.441088438034058, 'Backtrace speed': 0.0007762908935546875, 'Forward computations': 157788}
{'Filename': '0068.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.6, 'Word Count': 5, 'transcription': "pickled where's peck peck peppers", 'Viterbi transcription': "the pickled where's peck picked peppers the", 'Decode speed': 2.7081172466278076, 'Backtrace speed': 0.00045990943908691406, 'Forward computations': 96888}
{'Filename': '0069.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 2, 'WER': 0.6666666666666666, 'Word Count': 9, 'transcription': 'picked pickled a pickled peter of peppers peppers piper', 'Viterbi transcription': 'the pickled pickled of pickled peter of o

{'Filename': '0089.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 5, 'WER': 0.5294117647058824, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': "peck the of of the picked peck of pickled peppers where's the peck of picked a peppers peter piper picked the", 'Decode speed': 3.5759663581848145, 'Backtrace speed': 0.000865936279296875, 'Forward computations': 125658}
{'Filename': '0090.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.35294117647058826, 'Word Count': 17, 'transcription': "peter piper picked a peck of pickled peppers where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': "peck the peter piper picked a peck of pickled peppers where's the peck of pickled picked where's peter piper peck picked the", 'Decode speed': 4.7421488761901855, 'Backtrace speed': 0.0011775493621826172, 'Forward computations': 184668}
{'F

...

{'Filename': '0220.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.6666666666666666, 'Word Count': 9, 'transcription': "where's the peck of pickled peppers peter piper picked", 'Viterbi transcription': "the where's the of peck of pickled peter peter of piper of picked the", 'Decode speed': 2.621199131011963, 'Backtrace speed': 0.0004589557647705078, 'Forward computations': 96888}
{'Filename': '0221.wav', 'Insertions': 6, 'Deletions': 1, 'Substitutions': 2, 'WER': 0.5294117647058824, 'Word Count': 17, 'transcription': "peck a piper pickled peter of of the peck peter picked piper where's peppers pickled peppers picked", 'Viterbi transcription': "the peck of piper pickled peter of pickled of peck peter peck picked piper where's of where's the pickled peppers picked the", 'Decode speed': 5.577921628952026, 'Backtrace speed': 0.0010421276092529297, 'Forward computations': 209868}
{'Filename': '0222.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.5, 'W

{'Filename': '0242.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.2857142857142857, 'Word Count': 7, 'transcription': "where's the peck of peppers peter picked", 'Viterbi transcription': "where's of peck of peppers peter picked the", 'Decode speed': 2.064136505126953, 'Backtrace speed': 0.00043702125549316406, 'Forward computations': 77148}
{'Filename': '0243.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.7142857142857143, 'Word Count': 7, 'transcription': "where's the peck of peppers piper pickled", 'Viterbi transcription': "the where's picked of peck of peppers the piper pickled the", 'Decode speed': 2.1488351821899414, 'Backtrace speed': 0.0004832744598388672, 'Forward computations': 80928}
{'Filename': '0244.wav', 'Insertions': 7, 'Deletions': 0, 'Substitutions': 3, 'WER': 1.4285714285714286, 'Word Count': 7, 'transcription': 'peter picked a peck of pickled peppers', 'Viterbi transcription': "the of the the a of a the of peck of pickled whe

{'Filename': '0263.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.0, 'Word Count': 6, 'transcription': 'peter picked peck of pickled peppers', 'Viterbi transcription': "where's the the picked of peck peck of pickled peppers the", 'Decode speed': 2.8296914100646973, 'Backtrace speed': 0.0005178451538085938, 'Forward computations': 77148}
{'Filename': '0264.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.25, 'Word Count': 4, 'transcription': "where's peter piper peck", 'Viterbi transcription': "the the where's picked of piper peck the", 'Decode speed': 2.1901488304138184, 'Backtrace speed': 0.00039267539978027344, 'Forward computations': 61188}
{'Filename': '0265.wav', 'Insertions': 5, 'Deletions': 0, 'Substitutions': 2, 'WER': 2.3333333333333335, 'Word Count': 3, 'transcription': 'piper picked peppers', 'Viterbi transcription': "the picked a picked picked the where's the", 'Decode speed': 2.083406686782837, 'Backtrace speed': 0.000418424606323242

{'Filename': '0287.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 2, 'WER': 0.75, 'Word Count': 8, 'transcription': 'a peck of pickled peppers peter piper picked', 'Viterbi transcription': "the of peck of pickled the of where's peter piper picked the", 'Decode speed': 2.6210639476776123, 'Backtrace speed': 0.0006158351898193359, 'Forward computations': 84498}
{'Filename': '0288.wav', 'Insertions': 2, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.5, 'Word Count': 6, 'transcription': 'peppers pickled a peck picked peter', 'Viterbi transcription': 'the peppers pickled of peck picked peter the', 'Decode speed': 2.1888649463653564, 'Backtrace speed': 0.0005006790161132812, 'Forward computations': 80928}
{'Filename': '0289.wav', 'Insertions': 1, 'Deletions': 0, 'Substitutions': 1, 'WER': 0.6666666666666666, 'Word Count': 3, 'transcription': 'peter picked peppers', 'Viterbi transcription': "picked where's picked peppers", 'Decode speed': 1.5732231140136719, 'Backtrace speed': 0.000206

{'Filename': '0311.wav', 'Insertions': 9, 'Deletions': 0, 'Substitutions': 1, 'WER': 2.0, 'Word Count': 5, 'transcription': 'a peck of pickled peppers', 'Viterbi transcription': "of a where's the the of peck of pickled of where's piper the the", 'Decode speed': 4.265676736831665, 'Backtrace speed': 0.000759124755859375, 'Forward computations': 111378}
{'Filename': '0312.wav', 'Insertions': 4, 'Deletions': 0, 'Substitutions': 1, 'WER': 1.6666666666666667, 'Word Count': 3, 'transcription': "where's peter piper", 'Viterbi transcription': "picked where's peter the the a the", 'Decode speed': 2.0595321655273438, 'Backtrace speed': 0.0003848075866699219, 'Forward computations': 57618}
{'Filename': '0313.wav', 'Insertions': 9, 'Deletions': 1, 'Substitutions': 1, 'WER': 2.75, 'Word Count': 4, 'transcription': 'peter picked pickled peppers', 'Viterbi transcription': "picked the the a the pickled picked the picked where's piper the", 'Decode speed': 2.5063273906707764, 'Backtrace speed': 0.00053

In [3]:
# for wav_file in glob.glob('/group/teaching/asr/labs/recordings/*.wav'):    # replace path if using your own
#                                                                            # audio files
    
#     decoder = MyViterbiDecoder(f, wav_file)
    
#     decoder.decode()
#     (state_path, words) = decoder.backtrace()  # you'll need to modify the backtrace() from Lab 4
#                                                # to return the words along the best path
    
#     transcription = read_transcription(wav_file)
#     print(transcription)
#     print(words)
#     error_counts = wer.compute_alignment_errors(transcription, words)
#     word_count = len(transcription.split())
    
#     print(error_counts, word_count)     # you'll need to accumulate these to produce an overall Word Error Rate

In [4]:
# import glob
# import os
# import wer
# import observation_model
# import openfst_python as fst

# # ... (add your code to create WFSTs and Viterbi Decoder)
# import observation_model
# import math
# import openfst_python as fst

# class MyViterbiDecoder:
    
#     NLL_ZERO = 1e10  # define a constant representing -log(0).  This is really infinite, but approximate
#                      # it here with a very large number
    
#     def __init__(self, f, audio_file_name):
#         """Set up the decoder class with an audio file and WFST f
#         """
#         self.om = observation_model.ObservationModel()
#         self.f = f
        
#         if audio_file_name:
#             self.om.load_audio(audio_file_name)
#         else:
#             self.om.load_dummy_audio()
        
#         self.initialise_decoding()

        
#     def initialise_decoding(self):
#         """set up the values for V_j(0) (as negative log-likelihoods)
        
#         """
        
#         self.V = []   # stores likelihood along best path reaching state j
#         self.B = []   # stores identity of best previous state reaching state j
#         self.W = []   # stores output labels sequence along arc reaching j - this removes need for 
#                       # extra code to read the output sequence along the best path
        
#         for t in range(self.om.observation_length()+1):
#             self.V.append([self.NLL_ZERO]*self.f.num_states())
#             self.B.append([-1]*self.f.num_states())
#             self.W.append([[] for i in range(self.f.num_states())])  #  multiplying the empty list doesn't make multiple
        
#         # The above code means that self.V[t][j] for t = 0, ... T gives the Viterbi cost
#         # of state j, time t (in negative log-likelihood form)
#         # Initialising the costs to NLL_ZERO effectively means zero probability    
        
#         # give the WFST start state a probability of 1.0   (NLL = 0.0)
#         self.V[0][self.f.start()] = 0.0
        
#         # some WFSTs might have arcs with epsilon on the input (you might have already created 
#         # examples of these in earlier labs) these correspond to non-emitting states, 
#         # which means that we need to process them without stepping forward in time.  
#         # Don't worry too much about this!  
#         self.traverse_epsilon_arcs(0)        
        
#     def traverse_epsilon_arcs(self, t):
#         """Traverse arcs with <eps> on the input at time t
        
#         These correspond to transitions that don't emit an observation
        
#         We've implemented this function for you as it's slightly trickier than
#         the normal case.  You might like to look at it to see what's going on, but
#         don't worry if you can't fully follow it.
        
#         """
        
#         states_to_traverse = list(self.f.states()) # traverse all states
#         while states_to_traverse:
            
#             # Set i to the ID of the current state, the first 
#             # item in the list (and remove it from the list)
#             i = states_to_traverse.pop(0)   
        
#             # don't bother traversing states which have zero probability
#             if self.V[t][i] == self.NLL_ZERO:
#                     continue
        
#             for arc in self.f.arcs(i):
                
#                 if arc.ilabel == 0:     # if <eps> transition
                  
#                     j = arc.nextstate   # ID of next state  
                
#                     if self.V[t][j] > self.V[t][i] + float(arc.weight):
                        
#                         # this means we've found a lower-cost path to
#                         # state j at time t.  We might need to add it
#                         # back to the processing queue.
#                         self.V[t][j] = self.V[t][i] + float(arc.weight)
                        
#                         # save backtrace information.  In the case of an epsilon transition, 
#                         # we save the identity of the best state at t-1.  This means we may not
#                         # be able to fully recover the best path, but to do otherwise would
#                         # require a more complicated way of storing backtrace information
#                         self.B[t][j] = self.B[t][i] 
                        
#                         # and save the output labels encountered - this is a list, because
#                         # there could be multiple output labels (in the case of <eps> arcs)
#                         if arc.olabel != 0:
#                             self.W[t][j] = self.W[t][i] + [arc.olabel]
#                         else:
#                             self.W[t][j] = self.W[t][i]
                        
#                         if j not in states_to_traverse:
#                             states_to_traverse.append(j)

    
#     def forward_step(self, t):
          
#         for i in self.f.states():
            
#             if not self.V[t-1][i] == self.NLL_ZERO:   # no point in propagating states with zero probability
                
#                 for arc in self.f.arcs(i):
                    
#                     if arc.ilabel != 0: # <eps> transitions don't emit an observation
#                         j = arc.nextstate
#                         tp = float(arc.weight)  # transition prob
#                         ep = -self.om.log_observation_probability(self.f.input_symbols().find(arc.ilabel), t)  # emission negative log prob
#                         prob = tp + ep + self.V[t-1][i] # they're logs
#                         if prob < self.V[t][j]:
#                             self.V[t][j] = prob
#                             self.B[t][j] = i
                            
#                             # store the output labels encountered too
#                             if arc.olabel !=0:
#                                 self.W[t][j] = [arc.olabel]
#                             else:
#                                 self.W[t][j] = []
                            
    
#     def finalise_decoding(self):
#         """ this incorporates the probability of terminating at each state
#         """
        
#         for state in self.f.states():
#             final_weight = float(self.f.final(state))
#             if self.V[-1][state] != self.NLL_ZERO:
#                 if final_weight == math.inf:
#                     self.V[-1][state] = self.NLL_ZERO  # effectively says that we can't end in this state
#                 else:
#                     self.V[-1][state] += final_weight
                    
#         # get a list of all states where there was a path ending with non-zero probability
#         finished = [x for x in self.V[-1] if x < self.NLL_ZERO]
#         if not finished:  # if empty
#             print("No path got to the end of the observations.")
        
        
#     def decode(self):
#         self.initialise_decoding()
#         t = 1
#         while t <= self.om.observation_length():
#             self.forward_step(t)
#             self.traverse_epsilon_arcs(t)
#             t += 1
#         self.finalise_decoding()
    
#     def backtrace(self):
        
#         best_final_state = self.V[-1].index(min(self.V[-1])) # argmin
#         best_state_sequence = [best_final_state]
#         best_out_sequence = []
        
#         t = self.om.observation_length()   # ie T
#         j = best_final_state
        
#         while t >= 0:
#             i = self.B[t][j]
#             best_state_sequence.append(i)
#             best_out_sequence = self.W[t][j] + best_out_sequence  # computer scientists might like
#                                                                                 # to make this more efficient!

#             # continue the backtrace at state i, time t-1
#             j = i  
#             t-=1
            
#         best_state_sequence.reverse()
        
#         # convert the best output sequence from FST integer labels into strings
        
        
        
#         # convert the best output sequence from FST integer labels into strings
#         best_out_sequence = ' '.join([ self.f.output_symbols().find(label) for label in best_out_sequence])
        
        
#         output=[]
#         full_words = [x for x in lex.values()]
        
#         this_str=[]
        
#         for label in best_out_sequence.split(" "):
#             this_str.append(label)
            
#             if this_str in lex.values():
#                 for i in  lex:
                    
#                     if lex.get(i)==this_str:
#                         output+=[f"{i}"]
#                 this_str=[]
#         output = " ".join([n for n in output])
#         return (best_state_sequence, output)
# import openfst_python as fst

# def parse_lexicon(lex_file):
#     """
#     Parse the lexicon file and return it in dictionary form.
    
#     Args:
#         lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
#                         eg. peppers p eh p er z

#     Returns:
#         lex (dict): dictionary mapping words to list of phones
#     """
    
#     lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
#     with open(lex_file, 'r') as f:
#         for line in f:
#             line = line.split()  # split at each space
#             lex[line[0]] = line[1:]  # first field the word, the rest is the phones
#     return lex

# def generate_symbol_tables(lexicon, n=3):
#     '''
#     Return word, phone and state symbol tables based on the supplied lexicon
        
#     Args:
#         lexicon (dict): lexicon to use, created from the parse_lexicon() function
#         n (int): number of states for each phone HMM
        
#     Returns:
#         word_table (fst.SymbolTable): table of words
#         phone_table (fst.SymbolTable): table of phones
#         state_table (fst.SymbolTable): table of HMM phone-state IDs
#     '''
    
#     state_table = fst.SymbolTable()
#     phone_table = fst.SymbolTable()
#     word_table = fst.SymbolTable()
    
#     # add empty <eps> symbol to all tables
#     state_table.add_symbol('<eps>')
#     phone_table.add_symbol('<eps>')
#     word_table.add_symbol('<eps>')
    
#     for word, phones  in lexicon.items():
        
#         word_table.add_symbol(word)
        
#         for p in phones: # for each phone
            
#             phone_table.add_symbol(p)
#             for i in range(1,n+1): # for each state 1 to n
#                 state_table.add_symbol('{}_{}'.format(p, i))
            
#     return word_table, phone_table, state_table


# # call these two functions
# lex = parse_lexicon('lexicon.txt')
# word_table, phone_table, state_table = generate_symbol_tables(lex)
# def generate_phone_wfst(f, start_state, phone, n):
#     """
#     Generate a WFST representing an n-state left-to-right phone HMM.
    
#     Args:
#         f (fst.Fst()): an FST object, assumed to exist already
#         start_state (int): the index of the first state, assumed to exist already
#         phone (str): the phone label 
#         n (int): number of states of the HMM excluding start and end
        
#     Returns:
#         the final state of the FST
#     """
    
#     current_state = start_state
    
#     for i in range(1, n+1):
    
#         in_label = state_table.find('{}_{}'.format(phone, i))
        
#         # self-loop back to current state
#         f.add_arc(current_state, fst.Arc(in_label, 0, None, current_state))
        
#         # transition to next state
        
#         # we want to output the phone label on the final state
#         # note: if outputting words instead this code should be modified
#         if i == n:
#             out_label = phone_table.find(phone)
#         else:
#             out_label = 0   # output empty <eps> label
            
#         next_state = f.add_state()
#         f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_state))    
       
#         current_state = next_state
#     return current_state




# def create_wfst(n):
#     f = fst.Fst()
#     start_state = f.add_state()
#     f.set_start(start_state)

#     for word, phones in lex.items():
#         current_state = start_state
#         for phone in phones:
#             # Now, generate_phone_wfst does not output phone symbols.
#             current_state = generate_phone_wfst(f, current_state, phone, n, state_table)
        
#         word_symbol = word_table.find(word)
#         end_state = f.add_state()
#         f.add_arc(current_state, fst.Arc(0, word_symbol, None, end_state))
#         f.set_final(end_state)

#     f.set_input_symbols(state_table)
#     f.set_output_symbols(word_table)
#     return f




    


# def read_transcription(wav_file):
#     """
#     Get the transcription corresponding to wav_file.
#     """
    
#     transcription_file = os.path.splitext(wav_file)[0] + '.txt'
    
#     with open(transcription_file, 'r') as f:
#         transcription = f.readline().strip()
    
#     return transcription


# f = create_wfst(3)
# f.set_input_symbols(state_table)
# f.set_output_symbols(phone_table)
# print([x for x in lex.values()])
